In [3]:
!pip install stanza
!pip install scikit-learn # Install scikit-learn if you haven't already
!pip install gensim
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Data handling
import pandas as pd
import json
import os
import re
import itertools
import time

# Natural Language Processing
import nltk
import stanza
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Machine Learning
import numpy as np
import concurrent.futures
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec

# Joblib
from joblib import dump

# Google Colab
# from google.colab import files

# Visualization
import matplotlib.pyplot as plt

# Deep Learning
from keras import models, layers
from keras.layers import Dense, Dropout, LSTM, Conv1D, Bidirectional
from keras.utils import to_categorical
from keras.models import Sequential
import tensorflow as tf


In [8]:
print("hello world")

hello world


In [9]:
test_data = {}
voc = {}

for i in range(5):
    # Download and load cleaned reviews
    os.system(f'wget -q "https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/cleaned_reviews_rating_{i+1}.json"')
    with open(f"cleaned_reviews_rating_{i+1}.json", "r") as f:
        test_data[i+1] = json.load(f)

    # Download and load vocabulary
    os.system(f'wget -q "https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/voc_rating_{i+1}.json"')
    with open(f"voc_rating_{i+1}.json", "r") as f:
        voc[i+1] = json.load(f)

    # # Optional: print confirmation
    # print(f"Loaded data for rating {i+1}:")
    # print("Sample review data:", test_data[:1])  # Show first entry as a sample
    # print("Sample vocabulary:", dict(list(voc.items())[:5]))  # Show first 5 entries as a sample

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_reviews_rating_1.json'

In [10]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_train.json"
f = open('laroseda_train.json')
# returns JSON object as
# a dictionary
train_data = json.load(f)

# Closing file
f.close()

'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: 'laroseda_train.json'

In [ ]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_test.json"
f = open('laroseda_test.json')
# returns JSON object as
# a dictionary
test_data = json.load(f)

# Closing file
f.close()


In [ ]:
def prep_doc(doc):
    cleaned = list()
    for i in doc:
        rev = i['title'] + ' . ' + i['content']
        # rev=i['content']
        # rev = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', rev)
        rev = rev.lower()
        # rev = re.sub(r'\s+', ' ', rev)
        # rev = re.sub(r'(?<! )(?=[.,!?()"\'/:;/\\\[^\]])|(?<=[.,!?()"\'/:;/\\\[^\]])(?! )', r' ', rev)
        cleaned.append(rev)
    return cleaned


def read_data(set):
    pos_data = list()
    neg_data = list()
    for i in set:
        # print(i)
        match i['starRating']:
            case '1':
                neg_data.append(i)
            case '2':
                neg_data.append(i)
            case '4':
                pos_data.append(i)
            case '5':
                pos_data.append(i)
    return neg_data, pos_data


def read_docs():
    f = open('laroseda_train.json')
    # returns JSON object as
    # a dictionary
    train_data = json.load(f)
    f.close()

    f = open('laroseda_test.json')
    # returns JSON object as
    # a dictionary
    test_data = json.load(f)
    # Closing file
    f.close()

    train_neg, train_pos = read_data(train_data['reviews'])
    test_neg, test_pos = read_data(test_data['reviews'])

    train_pos_clean = prep_doc(train_pos)
    train_neg_clean = prep_doc(train_neg)
    test_pos_clean = prep_doc(test_pos)
    test_neg_clean = prep_doc(test_neg)

    return train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean

In [ ]:
#old
neg_data, pos_data = read_data()
#new
data_pos=test_data[4]+test_data[5]
data_neg=test_data[1]+test_data[2] #+test_data[3]

negative=neg_data+data_neg
positive=pos_data+data_pos

In [ ]:

data = positive+negative

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])